<a href="https://colab.research.google.com/github/pi-aire/dataminig_sampling/blob/main/TP_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre





In [101]:
import numpy as np
import scipy.special as sps
import math
import matplotlib
import pandas as pd
import random

In [ ]:
# Le dataframe
df = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del df[37]
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,21,23,25,27,29,31,34,36,38,40,42,44,47,48,50,52,54,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,51,52,54,56,58,60,62,64,66,68,70,72,74


## Question 1

In [108]:
def algoFrequences(df:pd.DataFrame,nb_pattern)-> tuple:
  #R:set = set()
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df.shape[0])
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    size = len(df.iloc[i].dropna())
    w[i] = math.pow(2,size)
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selctionne un motif 
    pattern = np.array(df.iloc[row].dropna())
    random_v = random.randint(1, len(pattern) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifes non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [109]:
algoFrequences(df,10)

[array([ 2,  4,  7,  9, 21, 29, 34, 38, 49, 58, 62]),
 array([ 9, 14, 20, 25, 27, 29, 32, 34, 36, 40, 46, 56, 60, 62, 64, 66, 71,
        73, 74]),
 array([ 1,  6,  8, 11, 14, 15, 17, 19, 21, 24, 25, 27, 34, 42, 44, 46, 51,
        54, 56, 58, 60, 62, 64, 66, 72]),
 array([ 2,  3,  5,  7,  9, 13, 15, 17, 19, 22, 23, 25, 28, 29, 34, 38, 40,
        42, 46, 48, 50, 52, 56, 58, 60, 62, 64, 66, 68, 70, 72, 75]),
 array([ 1, 14, 16, 17, 31, 34, 36, 50, 52, 56, 62, 64, 72]),
 array([ 1,  3,  5, 11, 14, 15, 17, 19, 22, 23, 28, 31, 34, 36, 40, 42, 45,
        46, 48, 50, 52, 55, 56, 58, 60, 62, 66, 72]),
 array([ 1,  3,  5,  7, 11, 13, 15, 17, 21, 24, 25, 27, 29, 36, 39, 44, 47,
        51, 56, 58, 60, 62, 65, 68, 71, 72, 74]),
 array([ 5,  7, 10, 12, 17, 23, 25, 29, 31, 34, 38, 40, 44, 48, 57, 60, 62,
        69, 70, 72]),
 array([20, 21, 27, 31, 34, 38, 42, 48, 51, 52, 56, 62, 66, 69, 70, 73]),
 array([ 1, 26, 34, 55, 66, 70, 75])]

## Question 2

In [106]:
def algoArea(df:pd.DataFrame,nb_pattern)-> tuple:
  R = []
  IsInR = set()
  
  # set les probas
  w = np.zeros(df[0].count())
  totalW = 0
  for i in range(1, len(w)):
    size = len(df.iloc[i].dropna())
    w[i] = size * (2 ** (size - 1))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    # On cherche la ligne
    random_row = random.uniform(0,totalW)
    v, row = 0,0
    for i in range(len(w)):
      if v > random_row:
        row = i - 1
        break
      v += w[i]
      
    # On set les probabilités de k (taille du motif)
    # On souhaite que le sous-ensemble est une taille calculée 
    # proportionnellement avec les tailles des datarecords (ligne) 
    ks = np.zeros(len(df.iloc[row].dropna()))
    totalK = 0
    for i in range(len(ks)):
        ks[i] = sps.binom(len(ks), i + 1)
        totalK += ks[i]
    random_kp = random.uniform(0, totalK)
    # On cherche la ligne
    k, v = 0,0
    for i in range(len(ks)):
      if v > random_kp:
        k = i - 1
        break
      v += ks[i]
    
    pattern = np.array(df.iloc[row])
    for i in range(len(pattern)- k):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [107]:
algoArea(df,10)

[array([ 1,  9, 11, 13, 15, 23, 31, 38, 42, 44, 47, 48, 52, 64, 70]),
 array([ 5,  7, 18, 22, 25, 27, 34, 36, 38, 40, 44, 46, 50, 52, 56, 70]),
 array([ 2,  3,  9, 14, 17, 19, 22, 29, 31, 34, 36, 40, 48, 54, 58, 60, 62,
        64, 66, 68]),
 array([ 1,  3,  9, 11, 15, 23, 28, 31, 34, 36, 40, 42, 48, 50, 54, 56, 62,
        64, 66, 72]),
 array([11, 14, 21, 24, 39, 40, 42, 44, 46, 52, 54, 56, 65]),
 array([ 5,  9, 11, 19, 22, 34, 38, 42, 44, 46, 52, 56, 60]),
 array([ 3,  7, 14, 15, 17, 29, 34, 39, 42, 45, 46, 51, 55, 64, 72]),
 array([ 1, 15, 21, 24, 25, 29, 32, 40, 58, 65, 73]),
 array([ 3,  7, 17, 25, 29, 34, 36, 42, 46, 50, 60, 66, 72]),
 array([ 3,  5,  7, 11, 15, 17, 20, 29, 34, 36, 56, 60, 62])]

##Question 3

In [97]:
def frequences(df, patterns):
  frequencesP = np.zeros(len(patterns),dtype=float)
  for i in range(df.shape[0]):
    indexs = np.zeros(len(patterns),dtype=int)
    for j in range(df.iloc[i].count()):
      if df.iloc[i][j] == None:
        break
      for ip in range(len(patterns)):
        #print(df.iloc[i][j],patterns[ip][indexs[ip]])
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            frequencesP[ip] += 1.
        #print(indexs)
  for i in range(len(frequencesP)):
    frequencesP[i] = (frequencesP[i])/float(df.shape[0])
  return frequencesP

In [110]:
patterns = algoFrequences(df,10)

In [111]:
frequences(df,patterns)

array([0.01251564, 0.00375469, 0.00406758, 0.0034418 , 0.00312891,
       0.00375469, 0.00031289, 0.00031289, 0.07165207, 0.00125156])

In [ ]:
def aire(df, patterns):
  aireP = np.zeros(len(patterns),dtype=int)
  for i in range(df.shape[0]):
    indexs = np.zeros(len(patterns),dtype=int)
    for j in range(df.iloc[i].count()):
      if df.iloc[i][j] == None:
        break
      for ip in range(len(patterns)):
        #print(df.iloc[i][j],patterns[ip][indexs[ip]])
        if (len(patterns[ip])!= indexs[ip] and 
            df.iloc[i][j] == patterns[ip][indexs[ip]]):
          indexs[ip] += 1
          if len(patterns[ip]) == indexs[ip]:
            aireP[ip] += 1
        #print(indexs)
  for i in range(len(aireP)):
    aireP[i] = aireP[i]*len(patterns[i])
  return aireP

In [ ]:
patterns = algoArea(df,10)

In [ ]:
aire(df,patterns)

array([455.,  88.,  51.,  42.,  72.,  68., 240.,  96., 777., 150.])

## Question 4

*Nous allons maintenant tester nos algorithmes avec les jeux de données suggérés dans le sujet.
Dans un premier temps, nous chargeons les datasets.*

In [ ]:
dataset_1 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del dataset_1[37]
dataset_1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,21,23,25,27,29,31,34,36,38,40,42,44,47,48,50,52,54,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,51,52,54,56,58,60,62,64,66,68,70,72,74


In [ ]:
frequences(df,algoFrequences(dataset_1,10))

array([0.00031289, 0.04192741, 0.00125156, 0.00312891, 0.01720901,
       0.00031289, 0.00031289, 0.0175219 , 0.00093867, 0.02878598])

In [ ]:
dataset_2 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/kosarak_sequences.txt", sep=" ",header=None,)
dataset_2 = dataset_2[0].str.split(' ', expand=True)
dataset_2.head()

In [88]:
dataset_3 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/MSNBC.txt", sep=" ",header=None,)
dataset_3 = dataset_3[0].str.split(' ', expand=True)
dataset_3.head() 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,6,-1,9,-1,4,-1,10,-1,3,-1,10,-1,5,-1,10,-1,4,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,14,-1,13,-1,7,-1,4,-1,7,-1,13,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1,-1,4,-1,7,-1,1,-1,10,-1,1,-1,2,-1,1,-1,6,-1,1,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,9,-1,7,-1,9,-1,7,-1,9,-1,12,-1,6,-1,10,-1,13,-1,14,-1,12,-1,3,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,4,-1,10,-1,2,-1,10,-1,12,-1,10,-1,7,-1,1,-1,10,-1,-2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [112]:
patterns_3 = algoArea(dataset_3,10)

In [113]:
frequences(dataset_3,patterns_3)

array([3.14564328e-05, 0.00000000e+00, 3.14564328e-05, 3.14564328e-05,
       0.00000000e+00, 9.43692985e-05, 4.40390060e-04, 3.14564328e-05,
       9.43692985e-05, 3.14564328e-05])

In [ ]:
dataset_4 = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/BIBLE.txt", sep=" ",header=None,)
dataset_4 = dataset_4[0].str.split(' ', expand=True)
dataset_4.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174
0,1,-1,2,-1,3,-1,4,-1,5,-1,6,-1,7,-1,8,-1,9,-1,10,-1,11,-1,12,-1,13,-1,14,-1,15,-1,16,-1,17,-1,18,-1,19,-1,20,-1,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,40,-1,41,-1,42,-1,43,-1,44,-1,45,-1,46,-1,47,-1,31,-1,43,-1,48,-1,45,-1,46,-1,49,-1,-2,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,31,-1,50,-1,51,-1,15,-1,52,-1,53,-1,54,-1,55,-1,56,-1,8,-1,57,-1,31,-1,56,-1,50,-1,14,-1,58,-1,59,-1,60,-1,61,-1,53,-1,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,69,-1,62,-1,70,-1,15,-1,71,-1,16,-1,72,-1,73,-1,72,-1,74,-1,75,-1,20,-1,4,-1,52,-1,76,-1,-2,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,77,-1,78,-1,51,-1,79,-1,18,-1,80,-1,22,-1,81,-1,52,-1,82,-1,46,-1,83,-1,20,-1,84,-1,33,-1,10,-1,85,-1,86,-1,73,-1,61,-1,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Question 5

### Etude emprique

## Question 6

On a constaté que lorsque l'on avait une ligne beaucoup plus grande que les autres, les algorithmes de fréquence et d'aire favorisent la ligne la plus grande pour créer des motifs. On souhaite trouver des motifs fréquents, donc on souhaite éviter des motifs qui s'applique seulement dans des grandes lignes. Nous avons trouver trois solutions:

*   Supprimer les grandes lignes du dataset
*   Réduire la taille des grandes lignes
*   Modifier le poids accorder à la ligne la plus grande


In [ ]:
dsetGL = pd.read_fwf("https://www.philippe-fournier-viger.com/spmf/datasets/kosarak_sequences.txt", sep=" ",header=None,)
df_gl= dsetGL[0].str.split(' ', expand=True)

## Cas où le dataset est chargé dans un df
On va calculer la moyenne du nombre d'item par ligne et en fonction de cela on suprimera les colonnes supérieurs à la taille moyenne du nombre d'item/ligne
On ne regarde pas toute les lignes on en regarde que 40% (on pourras changer)

In [ ]:
df_gl.shape 

(990002, 103)

In [ ]:
def resizeDataSet(df,percentage):
  shape = df.shape
  sum = 0
  #for index, row in df.iterrows():
  for _ in range(int(shape[0]*percentage)):
    #if random.uniform(0.,100.) < 5.:
    sum += shape[1] - df.iloc[random.randint(0,shape[0]-1)].isnull().sum()
  mean = sum/(shape[0]*percentage)
  for i in range(shape[1]-1,int(mean),-1):
    del df[i]
  return df

In [ ]:
resizeDataSet(dataset_3,0.4)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,6,-1,9,-1,4,-1,10,-1,3,-1,10,-1,5,-1,10,-1,4,-1,-2,None,None,None,None,None,None,None,None
1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,7,-1,13,-1,14,-1,13,-1,7
2,1,-1,4,-1,7,-1,1,-1,10,-1,1,-1,2,-1,1,-1,6,-1,1,-1,-2,None,None,None,None,None,None
3,9,-1,7,-1,9,-1,7,-1,9,-1,12,-1,6,-1,10,-1,13,-1,14,-1,12,-1,3,-1,-2,None,None
4,4,-1,10,-1,2,-1,10,-1,12,-1,10,-1,7,-1,1,-1,10,-1,-2,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31785,9,-1,13,-1,8,-1,3,-1,14,-1,9,-1,13,-1,3,-1,14,-1,9,-1,-2,None,None,None,None,None,None
31786,6,-1,7,-1,6,-1,7,-1,6,-1,7,-1,6,-1,7,-1,6,-1,-2,None,None,None,None,None,None,None,None
31787,1,-1,12,-1,1,-1,10,-1,1,-1,2,-1,11,-1,4,-1,11,-1,6,-1,11,-1,4,-1,1,-1,7
31788,1,-1,2,-1,12,-1,2,-1,7,-1,1,-1,12,-1,7,-1,4,-1,12,-1,-2,None,None,None,None,None,None


## Troisième solution
Text  a mettre richard

Dans cette troisième approche, on va faire une moyenne mobile de la valeur de poid

[Info pour blabla](https://www.educatim.fr/tq/co/Module_TQ_web/co/moyenne_glissante.html)

In [116]:
def algoFrequencesSolutionA(df:pd.DataFrame,nb_pattern)-> tuple:
  #R:set = set()
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df[0].count())
  totalW = 0
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,len(df.iloc[i]))
    if i >= 2:
      w[i] = (w[i] + w[i-1] + w[i-2])/ 3
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selctionne un motif 
    pattern = np.array(df.iloc[row])
    random_v = random.randint(1, len(df.iloc[row]) - 1 )
    for i in range(len(pattern)- random_v):
      pattern = np.delete(pattern, random.randint(0, len(pattern) - 1 ))
    # On ajoute seulement les motifes non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R